In [44]:
from NikeCA import QA, Snowflake

import configparser

config = configparser.ConfigParser()
config.read('config2.ini')

username = config['snowflake'].get('username')
warehouse = config['snowflake'].get('warehouse')
role = config['snowflake'].get('role')
database = config['snowflake'].get('database')

sf = Snowflake(username=username, warehouse=warehouse, role=role, database=database)


In [55]:

def dashboard_imp_summary(date_min: str = '1900-01-01', date_max: str = '9999-12-31', column_filters: list = []) -> str:
    """
    Generate a SQL query for a dashboard summary report with optional column filters.

    Args:
        date_min (str): The minimum activity end date to include in the report. Default is '1900-01-01'.
        date_max (str): The maximum activity end date to include in the report. Default is '9999-12-31'.
        column_filters (list): A list of column names to include in the query GROUP BY clause. Default is an empty list.

    Returns:
        str: A SQL query string.

    Raises:
        TypeError: If date_min, date_max, or column_filters is not a string or list, respectively.

    Example:
        >>> dashboard_imp_summary(date_min='2022-01-01', column_filters=['***'])
    """
    
    import configparser
    
    config = configparser.ConfigParser()
    config.read('pip.ini')
    
    date_begin = config['imp_summary'].get('begin_date')
    date_end = config['imp_summary'].get('date_end')
    date = config['imp_summary'].get('date')
    sales = config['imp_summary'].get('sales')
    units = config['imp_summary'].get('units')
    table = config['imp_summary'].get('table')
    column = config['imp_summary'].get('column')
    column_filter = config['imp_summary'].get('column_filter')
    column_flag = config['imp_summary'].get('column_flag')
    column_flag_answer = config['imp_summary'].get('column_flag_answer')
    aggs = config['imp_summary'].get('aggs')
    
    # Ensure column_filters is a list
    if not isinstance(column_filters, list):
        raise TypeError("column_filters must be a list")

    # Join column filters into comma-separated string, if any
    if column_filters:
        column_filters_str = ', '.join(column_filters)
        group_column_filters_str = f'''GROUP BY 
            {column_filters_str}'''
        column_filters_str += ', '
    else:
        column_filters_str = ''
        group_column_filters_str = ''
    
    query_dashboard = f"""
    SELECT
        {column_filters_str}
        MIN({date_begin}) AS MIN_{date_begin}
        , MAX({date_end}) AS MAX_{date_end}
        , {aggs}
    FROM 
        {table}
    WHERE
        {date} BETWEEN '{date_min}' AND '{date_max}'
        AND {column} = '{column_filter}'
        AND {column_flag} = '{column_flag_answer}'
    {group_column_filters_str} 
    UNION
    
    SELECT
        {column_filters_str}
        MIN({date_begin}) AS MIN_{date_begin}
        , MAX({date_end}) AS MAX_{date_end}
        , {aggs}
    FROM 
        {table}
    WHERE
        {date} BETWEEN '{date_min}' AND '{date_max}'
        AND {column} <> '{column_filter}'
        AND {column_flag} = '{column_flag_answer}'
    {group_column_filters_str}
    ;
    """


    return query_dashboard

In [46]:
query = dashboard_imp_summary(date_min = '2023-01-07', date_max = '2023-03-04')
print(query)


    SELECT
        
        MIN(ACTIVITY_END_DT) AS MIN_ACTIVITY_END_DT
        , MAX(ACTIVITY_END_DT) AS MAX_ACTIVITY_END_DT
        , COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY), 0) AS TOTAL_NET_SALES_UNITS_TY, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY), 0) AS TOTAL_NET_SALES_AMT_TY, COALESCE(SUM(CASE WHEN (CUSTOMER_PARTNER_CLASSIFICATION  IN ('SILVER','GOLD')) THEN  TOTAL_OH_UNITS_TY End), 0) AS TOTAL_OH_UNITS_TY, COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK), 0) AS TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK, COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK), 0) AS TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY_ROLLING_4WK), 0) AS TOTAL_NET_SALES_AMT_TY_ROLLING_4WK, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY_ROLLING_13WK), 0) AS TOTAL_NET_SALES_AMT_TY_ROLLING_13WK, COALESCE(SUM(TOTAL_OH_UNITS_TY), 0) AS TOTAL_OH_UNITS_TY
    FROM 
        NGP_DA_PROD.POS_T.POS_CHNL_MCCF_SC_TREND_AGG
    WHERE
        ACTIVITY_END_DT BETWEEN '2023-01-07' AND '2023-03-04'
        AND RETAI

In [58]:
df = sf.snowflake_pull(query)
print(df)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nike.okta.com/app/snowflake/exk3gsxh1ykZ4xX5b1t7/sso/saml?SAMLRequest=jZLdctowEIVfxaNeY8k2NEGDyRAoA9MkMIGkKXfCFqBallyt%2FMPbV5jQSS6SyZ1GOmf3W50d3DS59CpuQGgVo8AnyOMq0alQ%2Bxg9raeda%2BSBZSplUiseoyMHdDMcAMtlQUelPahH%2FrfkYD1XSAFtH2JUGkU1AwFUsZwDtQldje7vaOgTWhhtdaIlemP53MEAuLGO8GJJQTi8g7UFxbiua7%2BOfG32OCSEYNLHTnWSfLvoGzfTB%2FoAk%2B5J7xROvnxluxXq%2FAWfYW3PIqCz9XrZWS5Wa%2BSNLqhjraDMuVlxU4mEPz3enQHAETzMf%2F7wQel6J1nGE50XpXWFfHfCO55iqffCzTqfxKjIRFqZ77NZ0tPZc3q7GE8X8rra6qR%2FFbJf1e%2FNy4b9Yf0g6umgIQnyni9hhqcw5wAln6tThNZdkTDqkG6HRGtCaDekhPhBGG2QN3ERCsVs67xwKpFxX2eWtWSsKPB%2FaMybLNpDcwiO2abbvPS2gb3CABqf4kTnDaFtdzP8wtwD%2FNbwul8P7svnk6WWIjl6U21yZj9OJPCD9kaknV0rpTxnQo7S1HAAl4yUuh4bzqxbY2tKjvDw3PX9Ig%2F%2FAQ%3D%3D&RelayState=58158 to authenticate

In [48]:
column_filters = ['ACTIVITY_END_DT, RETAILER_GEOGRAPHY_DISPLAY_NM, RETAILER_GEOGRAPHY_ID, CATEGORY, CHANNEL_DESC, \
DIVISION_DESC, STORE_ACCOUNT_TYPE_CD, EXPRESS_LANE, LICENSED_IND, PRICE_REALIZATION_BUCKET']

# column_filters = 'ACTIVITY_END_DT, RETAILER_GEOGRAPHY_ID, MSRP_USD, RETAILER_SALES_TYPE_DESC, CUSTOMER_TYPE, CUSTOMER_SUBTYPE_NAME, CUSTOMER_PARTNER_CLASSIFICATION, SEASON_YR_CD, CURR_WK_PRYR_END_DT, STORE_ACCOUNT_TYPE_CD, STORE_SALESMETHOD_TYPE_CD, CHANNEL_DESC, RETAILER_GEOGRAPHY_DISPLAY_NM, CATEGORY, ATHLETE_FIRST_NM, ATHLETE_LAST_NM, DIVISION_CD, DIVISION_DESC, FRANCHISE, GLOBAL_CATEGORY_CORE_FOCUS_CD, GLOBAL_CATEGORY_CORE_FOCUS_DESC, GLOBAL_CATEGORY_SUMMARY_DESC, LEAGUE_DESC, LICENSED_IND, MERCHANDISING_CLASSIFICATION_DESC, MODEL_GROUPS, MODEL_ID, MODEL_NM, PARENT_STYLE_CD, PRODUCT_CD, CATEGORY_DESC, PRODUCT_COLOR_COMBINATION_SHORT_DESC, SUB_CATEGORY_DESC, PRODUCT_TIER_DESC, SILHOUETTE_CD, SILHOUETTE_DESC, STYLE_DISPLAY_CD, STYLE_NM, SUB_CLASS_NM, SUB_FRANCHISE, TEAM_NM, POWER_ITEMS_NM, POWER_CATEGORY_DESC, POWER_FRANCHISE_NM, PRODUCT_OFFER_STATUS, PRODUCT_OFFER_STATUS_PLANNED, PRODUCT_OFFER_STATUS_PLANNED_SESN_YR_CD, EXPRESS_LANE, TYLY_FLAG, PRODUCT_PLANNING_DIVISION_NAME, PRODUCT_PLANNING_AGE_GENDER_NAME, PRODUCT_PLANNING_INTENDED_USE_NAME, PLANNING_PERFORMANCE_ACTIVITY_NAME, PRODUCT_PLANNING_STRUCTURE_NAME, PRODUCT_PLANNING_SILHOUETTE_NAME, PRODUCT_PLANNING_SUB_SILHOUETTE_NAME, PLANNING_PERFORMANCE_ACTIVITY_GROUP_NAME, PLANNING_FRANCHISE_PLANNING_PROGRAM_NAME, CONSUMER_CONSTRUCT_GLOBAL_CONSUMER_OFFENSE_NAME, CONSUMER_CONSTRUCT_DIMENSION_NAME, FAMILY, SUB_FAMILY, CONSUMER_CONSTRUCT_SUB_DIMENSION_NAME, PRODUCT_ACTIVE_IND, CAL_YEAR, QUARTER, MONTH'
# column_filters = column_filters.split(', ')
# column_filters

In [59]:
query = dashboard_imp_summary(date_min = '2023-01-07', date_max = '2023-03-04', column_filters=column_filters)
print(query)


    SELECT
        ACTIVITY_END_DT, RETAILER_GEOGRAPHY_DISPLAY_NM, RETAILER_GEOGRAPHY_ID, CATEGORY, CHANNEL_DESC, DIVISION_DESC, STORE_ACCOUNT_TYPE_CD, EXPRESS_LANE, LICENSED_IND, PRICE_REALIZATION_BUCKET, 
        MIN(ACTIVITY_END_DT) AS MIN_ACTIVITY_END_DT
        , MAX(ACTIVITY_END_DT) AS MAX_ACTIVITY_END_DT
        , COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY), 0) AS TOTAL_NET_SALES_UNITS_TY, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY), 0) AS TOTAL_NET_SALES_AMT_TY, COALESCE(SUM(CASE WHEN (CUSTOMER_PARTNER_CLASSIFICATION  IN ('SILVER','GOLD')) THEN  TOTAL_OH_UNITS_TY End), 0) AS TOTAL_OH_UNITS_TY, COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK), 0) AS TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK, COALESCE(SUM(TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK), 0) AS TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY_ROLLING_4WK), 0) AS TOTAL_NET_SALES_AMT_TY_ROLLING_4WK, COALESCE(SUM(TOTAL_NET_SALES_AMT_TY_ROLLING_13WK), 0) AS TOTAL_NET_SALES_AMT_TY_ROLLING_13WK
    FROM 
        NGP_D

In [57]:
df = sf.snowflake_pull(query)
print(df)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nike.okta.com/app/snowflake/exk3gsxh1ykZ4xX5b1t7/sso/saml?SAMLRequest=jZLdctowEIVfxaNe25IxmYAGyNAAg9v8UEM6be6ELUC1LDlaGZs8fYQJneQimdxppHN2v9XZwVVTSG%2FPDQithigMCPK4SnUm1HaIHlYzv4c8sExlTGrFh%2BjAAV2NBsAKWdJxZXcq4U8VB%2Bu5Qgpo%2BzBElVFUMxBAFSs4UJvS5fj2hnYCQkujrU61RG8snzsYADfWEZ4tGQiHt7O2pBjXdR3UUaDNFncIIZj0sVMdJd%2FO%2BsbN9IE%2BxKR71DuFky9e2b4LdfqCz7DWJxHQ%2BWq18Bf3yxXyxmfUa62gKrhZcrMXKX9Ibk4A4Aju4p%2FTAJSuN5LlPNVFWVlXKHAnvOEZlnor3KzxZIjKXGS2l8tsl6iLX83zvGzWf6c%2Fktk%2B3yzm%2F55Eze6fL%2BNkPa179fI2Rd7vc5idY5gxQMVjdYzQuivSiXzS9Um0IoR2QxpFQb%2FXf0TexEUoFLOt88ypRM4DnVvWkrGyxP%2BhMW%2FyaAvNLjzkj93mz8U6tJcYQONjnOi0IbTtbkZfmHuA3xpe9%2BvOfXk8WWgp0oM306Zg9uNEwiBsb0Tmb1op5QUTcpxlhgO4ZKTU9bXhzLo1tqbiCI9OXd8v8ugF&RelayState=58047 to authenticate..

In [60]:
df[
    (df['RETAILER_GEOGRAPHY_DISPLAY_NM']=='APLA') 
    & (df['DIVISION_DESC']=='FOOTWEAR DIVISION') 
    & (df['EXPRESS_LANE']=='NOT EXPRESSLANE')
    & (df['CATEGORY']=='NIKE SPORTSWEAR')
    & (df['LICENSED_IND']=='N')
].groupby([
    'RETAILER_GEOGRAPHY_DISPLAY_NM'
    , 'DIVISION_DESC'
    , 'EXPRESS_LANE'
    , 'CATEGORY'
    , 'LICENSED_IND'
    , 'PRICE_REALIZATION_BUCKET'
])[[
    'TOTAL_NET_SALES_UNITS_TY'
    , 'TOTAL_NET_SALES_AMT_TY'
]].sum().sort_values(by='TOTAL_NET_SALES_AMT_TY', ascending=False)



TOTAL_NET_SALES_UNITS_TY  \
RETAILER_GEOGRAPHY_DISPLAY_NM DIVISION_DESC     EXPRESS_LANE    CATEGORY        LICENSED_IND PRICE_REALIZATION_BUCKET                            
APLA                          FOOTWEAR DIVISION NOT EXPRESSLANE NIKE SPORTSWEAR N            FP                                    3028874.000   
                                                                                             MD                                    1410194.000   
                                                                                             CL                                     373018.000   
                                                                                             UA                                       1666.000   

                                                                                                                      TOTAL_NET_SALES_AMT_TY  
RETAILER_GEOGRAPHY_DISPLAY_NM DIVISION_DESC     EXPRESS_LANE    CATEGORY        LICENSED_IND PRICE_REALIZATION_BUCKET                         
APLA                          FOOTWEAR DIVISION NOT EXPRESSLANE NIKE SPORTSWEAR N            FP                                273701338.119  
                                                                                             MD                                 81025555.787  
                                                                                             CL                                 14894570.075  
                                                                                             UA                                    22312.238

In [ ]:

column_filters = ['ACTIVITY_END_DT, RETAILER_GEOGRAPHY_DISPLAY_NM, RETAILER_GEOGRAPHY_ID, CATEGORY, CHANNEL_DESC, \
DIVISION_DESC, STORE_ACCOUNT_TYPE_CD, EXPRESS_LANE, LICENSED_IND, PRICE_REALIZATION_BUCKET']

In [61]:
import polars as pl

In [62]:
df2 = pl.from_pandas(df)

In [66]:
df2

ACTIVITY_END_DT,RETAILER_GEOGRAPHY_DISPLAY_NM,RETAILER_GEOGRAPHY_ID,CATEGORY,CHANNEL_DESC,DIVISION_DESC,STORE_ACCOUNT_TYPE_CD,EXPRESS_LANE,LICENSED_IND,PRICE_REALIZATION_BUCKET,MIN_ACTIVITY_END_DT,MAX_ACTIVITY_END_DT,TOTAL_NET_SALES_UNITS_TY,TOTAL_NET_SALES_AMT_TY,TOTAL_OH_UNITS_TY,TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK,TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK,TOTAL_NET_SALES_AMT_TY_ROLLING_4WK,TOTAL_NET_SALES_AMT_TY_ROLLING_13WK
str,str,i8,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64
"""2023-02-04""","""GREATER CHINA""",79,"""YOUNG ATHLETES…","""CATEGORY SPECI…","""APPAREL DIVISI…","""WHOLESALE""","""NOT EXPRESSLAN…","""N""","""UA""","""2023-02-04""","""2023-02-04""",0.0,0.0,0.0,0.0,2.0,22.668,30.178
"""2023-01-21""","""GREATER CHINA""",79,"""NIKE BASKETBAL…","""NIKE COMPANY S…","""APPAREL DIVISI…","""DTC""","""UPDATE""","""N""","""MD""","""2023-01-21""","""2023-01-21""",53.0,2983.545,0.0,146.0,153.0,8624.155,9061.57
"""2023-01-14""","""GREATER CHINA""",79,"""NIKE SPORTSWEA…","""SPORTING GOODS…","""APPAREL DIVISI…","""WHOLESALE""","""UPDATE""","""N""","""UA""","""2023-01-14""","""2023-01-14""",0.0,0.0,0.0,0.0,0.0,0.0,-28.759
"""2023-01-07""","""GREATER CHINA""",79,"""GOLF""","""NIKE STORES OW…","""APPAREL DIVISI…","""DTC""","""NOT EXPRESSLAN…","""N""","""FP""","""2023-01-07""","""2023-01-07""",17.0,416.638,0.0,79.0,192.0,1846.356,4560.189
"""2023-01-07""","""GREATER CHINA""",79,"""OTHER""","""NIKE FACTORY S…","""EQUIPMENT DIVI…","""DTC""","""NOT EXPRESSLAN…","""N""","""MD""","""2023-01-07""","""2023-01-07""",3.0,131.56,0.0,3.0,3.0,131.56,131.56
"""2023-01-28""","""GREATER CHINA""",79,"""WOMENS TRAININ…","""SPORTING GOODS…","""APPAREL DIVISI…","""WHOLESALE""","""NOT EXPRESSLAN…","""N""","""MD""","""2023-01-28""","""2023-01-28""",200.0,7181.921,0.0,684.0,1733.0,24047.464,59300.774
"""2023-01-21""","""GREATER CHINA""",79,"""JORDAN BRAND""","""NIKE DC UN-ASS…","""FOOTWEAR DIVIS…","""UNKNOWN""","""FULFILL""","""N""","""UA""","""2023-01-21""","""2023-01-21""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""2023-01-14""","""GREATER CHINA""",79,"""GOLF""","""NIKE FACTORY S…","""EQUIPMENT DIVI…","""DTC""","""NOT EXPRESSLAN…","""N""","""MD""","""2023-01-14""","""2023-01-14""",0.0,0.0,0.0,0.0,2.0,0.0,80.46
"""2023-03-04""","""NORTH AMERICA""",69,"""OTHER""","""NIKE.COM""","""APPAREL DIVISI…","""DTC""","""NOT EXPRESSLAN…","""N""","""UA""","""2023-03-04""","""2023-03-04""",3585.0,148465.73,0.0,14515.0,52216.0,576553.0,2072505.3


In [67]:
df

,ACTIVITY_END_DT,RETAILER_GEOGRAPHY_DISPLAY_NM,RETAILER_GEOGRAPHY_ID,CATEGORY,CHANNEL_DESC,DIVISION_DESC,STORE_ACCOUNT_TYPE_CD,EXPRESS_LANE,LICENSED_IND,PRICE_REALIZATION_BUCKET,MIN_ACTIVITY_END_DT,MAX_ACTIVITY_END_DT,TOTAL_NET_SALES_UNITS_TY,TOTAL_NET_SALES_AMT_TY,TOTAL_OH_UNITS_TY,TOTAL_NET_SALES_UNITS_TY_ROLLING_4WK,TOTAL_NET_SALES_UNITS_TY_ROLLING_13WK,TOTAL_NET_SALES_AMT_TY_ROLLING_4WK,TOTAL_NET_SALES_AMT_TY_ROLLING_13WK
0,2023-02-04,GREATER CHINA,79,YOUNG ATHLETES,CATEGORY SPECIALTY,APPAREL DIVISION,WHOLESALE,NOT EXPRESSLANE,N,UA,2023-02-04,2023-02-04,0.000,0.000,0.000,0.000,2.000,22.668,30.178
1,2023-01-21,GREATER CHINA,79,NIKE BASKETBALL,NIKE COMPANY STORE,APPAREL DIVISION,DTC,UPDATE,N,MD,2023-01-21,2023-01-21,53.000,2983.545,0.000,146.000,153.000,8624.155,9061.570
2,2023-01-14,GREATER CHINA,79,NIKE SPORTSWEAR,SPORTING GOODS,APPAREL DIVISION,WHOLESALE,UPDATE,N,UA,2023-01-14,2023-01-14,0.000,0.000,0.000,0.000,0.000,0.000,-28.759
3,2023-01-07,GREATER CHINA,79,GOLF,NIKE STORES OWNED,APPAREL DIVISION,DTC,NOT EXPRESSLANE,N,FP,2023-01-07,2023-01-07,17.000,416.638,0.000,79.000,192.000,1846.356,4560.189
4,2023-01-07,GREATER CHINA,79,OTHER,NIKE FACTORY STORES,EQUIPMENT DIVISION,DTC,NOT EXPRESSLANE,N,MD,2023-01-07,2023-01-07,3.000,131.560,0.000,3.000,3.000,131.560,131.560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103093,2023-01-28,APLA,76,MENS TRAINING,DEPARTMENT STORE,FOOTWEAR DIVISION,WHOLESALE,NOT EXPRESSLANE,N,CL,2023-01-28,2023-01-28,135.000,4335.847,0.000,410.000,1787.000,14732.494,56126.789
103094,2023-02-11,NORTH AMERICA,69,GOLF,VALUE CHAIN,APPAREL DIVISION,WHOLESALE,NOT EXPRESSLANE,N,UA,2023-02-11,2023-02-11,0.000,0.000,0.000,0.000,0.000,0.000,0.000
103095,2023-02-18,NORTH AMERICA,69,NIKE BASKETBALL,NIKE FACTORY STORES,EQUIPMENT DIVISION,DTC,NOT EXPRESSLANE,Y,CL,2023-02-18,2023-02-18,1.000,6.990,0.000,7.000,101.000,49.600,1390.060
103096,2023-01-14,APLA,76,NIKE BASKETBALL,SPORTING GOODS,EQUIPMENT DIVISION,WHOLESALE,NOT EXPRESSLANE,N,UA,2023-01-14,2023-01-14,0.000,0.000,0.000,2.000,23.000,5.315,4.330
